In [1]:
import pandas as pd
import sys
sys.path.append('../../pytorch-transformer/')

In [4]:
# 엑셀파일 -> csv 변환
xls_a = pd.read_excel('../data/korean-english-corpus/1_spoken1.xlsx', index_col=None)
xls_b = pd.read_excel('../data/korean-english-corpus/1_spoken2.xlsx', index_col=None)

xls_a.to_csv('../data/korean-english-corpus/1_spoken1.csv', encoding='utf-8', index=False)
xls_b.to_csv('../data/korean-english-corpus/1_spoken2.csv', encoding='utf-8', index=False)

In [2]:
# CSV로 변환한 말뭉치 파일 로드 및 합병
# 총 400,000개의 병렬 문장 쌍
data_a = pd.read_csv('../data/korean-english-corpus/1_spoken1.csv', encoding='utf-8')
data_b = pd.read_csv('../data/korean-english-corpus/1_spoken2.csv', encoding='utf-8')

data = pd.concat([data_b, data_a], ignore_index=True)

In [28]:
data

,SID,원문,번역문
0,200001.0,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t..."
1,200002.0,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...
2,200003.0,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version."
3,200004.0,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ..."
4,200005.0,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra..."
...,...,...,...
399995,199996.0,나는 먼저 청소기로 바닥을 밀었어요.,"First of all, I vacuumed the floor."
399996,199997.0,나는 먼저 팀 과제를 하고 놀러 갔어요.,I did the team assignment first and went out t...
399997,199998.0,나는 비 같은 멋진 연예인을 좋아해요.,I like cool entertainer like Rain.
399998,199999.0,나는 멋진 자연 경치를 보고 눈물을 흘렸어.,I cried seeing the amazing scenery.


In [8]:
# # SID column 삭제
# kor_eng = data.drop(['SID'], axis = 1)
# kor_eng.to_csv('../data/kor_eng.csv', encoding='utf-8', index=False)

In [4]:
test_data = data[60102:90102]
# test_data
test_data = test_data.drop(['SID'], axis = 1)
test_data.to_csv('../data/kor_eng_short3.csv', encoding = 'utf-8', index=False)

,SID,원문,번역문
0,200001.0,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t..."
1,200002.0,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...
2,200003.0,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version."
3,200004.0,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ..."
4,200005.0,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra..."
...,...,...,...
60097,260098.0,저의 이름은 타쿠치입니다.,My name is Takeuchi.
60098,260099.0,저의 이름의 본뜻은 넓게 편다는 뜻을 가지고 있습니다.,The original meaning of my name means spreadin...
60099,260100.0,저의 자녀들은 언제나 늦게 일어납니다.,My children always wake up late.
60100,260101.0,저의 작업은 제 웹사이트를 통해 확인 가능합니다.,You can see my works through my website.


### Test Set 파일 생성

In [23]:
t1 = data[:30]
t2 = data[100:141]
t3 = data[30000:30032]
test_data2 = pd.concat([data[:30], data[100:140]], ignore_index=True)
test_data3 = pd.concat([test_data2, data[30000:30030]], ignore_index=True)
test_data3
test_data3.to_csv('../data/kor_eng_test2.csv', encoding = 'utf-8', index=False)

## Wordpiece Tokenizer

In [39]:
import os
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)
path = '../data/kor_eng_short1_short_2.csv'
# path = '../data/test.csv'

corpus_file   = [path]  # data path
vocab_size    = 32000   #vocab의 크기. 보통 32,000이 좋다고 알려짐.
limit_alphabet= 6000    #merge 수행 전 initial tokens이 유지되는 숫자 제한
output_path   = 'hugging_%d'%(vocab_size)
min_frequency = 5  # 단어의 최소 발생 빈도

tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet, 
               show_progress=True)

# save tokenizer
hf_model_path='tokenizer_model_short3'
if not os.path.isdir(hf_model_path):

    os.mkdir(hf_model_path)
tokenizer.save_model(hf_model_path)  # vocab.txt 파일 한개가 만들어진다

['tokenizer_model_short3/vocab.txt']

### Ensemble Model vocab 생성을 위한 훈련데이터 셋 concat

In [29]:
data

,SID,원문,번역문
0,200001.0,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t..."
1,200002.0,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...
2,200003.0,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version."
3,200004.0,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ..."
4,200005.0,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra..."
...,...,...,...
399995,199996.0,나는 먼저 청소기로 바닥을 밀었어요.,"First of all, I vacuumed the floor."
399996,199997.0,나는 먼저 팀 과제를 하고 놀러 갔어요.,I did the team assignment first and went out t...
399997,199998.0,나는 비 같은 멋진 연예인을 좋아해요.,I like cool entertainer like Rain.
399998,199999.0,나는 멋진 자연 경치를 보고 눈물을 흘렸어.,I cried seeing the amazing scenery.


In [36]:
train = pd.concat([data[0:30000], data[30102:60102]], ignore_index=True)
train.to_csv('../data/kor_eng_short1_short_2.csv', encoding = 'utf-8', index=False)